In [18]:

# Generación a del algoritmo de PRNG básico (el de generador de numeros pseudoaleatorios)

import random

def generar_keystream(seed: str, longitud: int) -> bytes:
    # Passing the current time as the seed value

    # En base a la libreriad e random (se utiliza una semilla para que genere el mismo cifrado.) que sea deterministico
    random.seed(seed)

    # Se crea un arreglo de bytes para poder almacenar. Es el mismo de list pero osea no consumis tanto espacio 
    arreglodebytes = bytearray()
    for _ in range(longitud):
        numero = random.randint(0, 255)  #Se elige de 8 bytes o pues 256 bits los número 
        arreglodebytes.append(numero)

    # regrsa el arreglo a bytes
    return bytes(arreglodebytes)
    


valor_a_investigar = "HyperPlay"
seed = "Estoesunasemilla"

print(generar_keystream( seed ,len(valor_a_investigar)))

#Debe de regresar la misma keystream
print(generar_keystream( seed ,len(valor_a_investigar)))



b'\xc7\t\xe4\xado\xc4\x05\x97\x91'
b'\xc7\t\xe4\xado\xc4\x05\x97\x91'


1.2 Función de cifrado 

En base a la instrucción vista en el documento se debe de genrar una función que pueda cifrar el mensaje de texto plano usando XOR con el keystream entonces ya que tenemos el keystream generado en el inciso anterior, ya podemos hacer el cifrado. 

In [19]:

# funcion pasada para hacer XOR del ejercicio anterior prompt: funcion XOR donde si es 1 o 0 o viceversa es 1 y los demás son cero tambien necesito hacer padding para que pueda ser del mismo tamaño. 


def cifrar(mensaje:str, clave:str) -> bytes:
    # args mensaje que se desea encriptar
    #clave es el keystream generado anteriormente. 

    mensaje = mensaje.encode("utf-8")
    clave = generar_keystream( clave ,len(mensaje))


    # prompt utilizado para averiguar como hacer XOR en bytes 
    # How can I XOR two bytes objects of the same length in Python and return the result as bytes?
    mensaje_cifrado = bytes(m ^ k for m, k in zip(mensaje, clave))


    return mensaje_cifrado


mensaja_a_cifrar ="hola mundo" 
cifrado = cifrar(mensaja_a_cifrar,"valorclave" )

print(cifrado)




    

b'7\x19+h\xd0oJ\xedt\xb8'


1.3 FUnción de descifrado.

En este apartado se va a implementar una función que descifre el mensaje cifrado del inciso anterior. 

Los principales requisitos son los siguientes
Este tiene que aceptar el texto cifrado y clave
genere el mismo keystream usado en el cifrado
aplicar XOR para recuperar el mensaje orignial
Verifique que el descifrado reproduce exactamente el texto plano. original.


In [20]:
def descifrar(mensaje:bytes, clave:str) -> str:
    # args mensaje que se desea encriptar
    #clave es el keystream generado anteriormente. 

    clave = generar_keystream( clave ,len(mensaje))

    # prompt utilizado para averiguar como hacer XOR en bytes 
    # How can I XOR two bytes objects of the same length in Python and return the result as bytes?
    mensaje_descifrado = bytes(m ^ k for m, k in zip(mensaje, clave))



    mensaje = mensaje_descifrado.decode("latin-1")

    return mensaje



if mensaja_a_cifrar == descifrar(cifrado,"valorclave" ):
    print("mensaje correcto imprimer el valor: ",descifrar(cifrado,"valorclave" ) )


mensaje correcto imprimer el valor:  hola mundo


2.1 Variación de la Clave (5 puntos)
o ¿Qué sucede cuando cambia la clave utilizada para generar el keystream? Demuestre con un
ejemplo concreto.



In [21]:


#En este caso si fuera cifrarlo con esta clave pero se cambia la clave por otra, entonces al momento de descifrar con una clave diferente no es el mensaje exacto como se muestra en el ejemplo anterior. 
clave = "manchilld"

mensaja_a_cifrar ="fueras de juego" 
cifrado = cifrar(mensaja_a_cifrar, clave )

print(cifrado)

clave2 = "nchold"

print(descifrar(cifrado, clave2 ))

b'\xf9~\x1fG\xde\xe2\x16x\x95:\x9c\xd3W;\x19'
¡ÞW7#ÐøB[lE)£


En este caso si fuera cifrarlo con esta clave pero se cambia la clave por otra, entonces al momento de descifrar con una clave diferente no es el mensaje exacto como se muestra en el ejemplo anterior. 

En este caso el mensaje "fueras de juego" se cifra usando la clave "manchilld". Luego se intenta descifrar el resultado usando una clave diferente nchold pero la clave no es la misma, el keystream generado será distinto. Por lo tanto, el resultado del descifrado no será el mensaje original, demostrando que la clave debe ser exactamente la misma para poder recuperar correctamente la información.


## .2 Reutilización del Keystream (5 puntos)
¿Qué riesgos de seguridad existen si reutiliza el mismo keystream para cifrar dos mensajes
diferentes? Implemente un ejemplo que demuestre esta vulnerabilidad.
Sugerencia: Cifre dos mensajes con la misma clave y analice qué información puede extraer un
atacante que intercepte ambos textos cifrados.

In [22]:

clave = "manchild"

mensaja_a_cifrar ="prueba de fuego" 
cifrado = cifrar(mensaja_a_cifrar, clave )

print(cifrado)

print(descifrar(cifrado, clave ))



mensaja_a_cifrar ="fueras de juego" 
cifrado = cifrar(mensaja_a_cifrar, clave )

print(cifrado)

print(descifrar(cifrado, clave ))

b'[`\r\xc51\xc1p]C\x069\xfc\xb0k1'
prueba de fuego
b'Mg\x1d\xd22\xd3p]C\x065\xfc\xb0k1'
fueras de juego


Si se usa el mimso keystream para cifrar diferentes mensajes en un cifrado de flujo esto es vulnerable ya que el algoritmo genera exactamente el mismo flujo de bytes, lo que puede observarse cuando aparecen secuencias iguales en los textos cifrados, como por ejemplo patrones repetidos en bytes (\xfc\xb0k1). Esto indica que parte del keystream se está reutilizando. Si un atacante intercepta dos mensajes cifrados con el mismo keystream, puede combinarlos entre sí y eliminar el efecto de la clave, obteniendo información directa sobre la relación entre los mensajes originales. A partir de esa información, se deduce que el contenido. 

¿Cómo afecta la longitud del keystream a la seguridad del cifrado? Considere tanto keystreams
más cortos como más largos que el mensaje.

La longitud del keystream influye en cómo se aplica el cifrado al mensaje. Si el keystream es más corto que el mensaje, no será suficiente para cifrar todo el contenido, por lo que el sistema necesitaría extenderlo o repetirlo para completar el proceso; si esto no se hace correctamente, puede debilitar la seguridad al generar patrones predecibles. En cambio, si el keystream es igual o más largo que el mensaje, cada parte del mensaje puede cifrarse con una porción distinta del flujo, lo que permite una protección más uniforme. En términos generales, un keystream que cubra completamente el mensaje proporciona mejores condiciones de seguridad que uno insuficiente, ya que evita limitaciones estructurales en el proceso de cifrado. 

2.4 Consideraciones Prácticas (5 puntos)
¿Qué consideraciones debe tener al generar un keystream en un entorno de producción real?
Mencione al menos 3 aspectos críticos.


La primera consideración que pensaría que deberia de usarse para un entorno de producción real seria el uso de un generador criptográficamente seguro porque el keystream debe generarse con un algoritmo aprobado, ya que nuestro generador es débil o predecible lo cual permite a un atacante pueda descifrar la información. Por otro lado, se considera que un único uso de claves y nunces ya que las claves deben ser únicas y luego desecharlos o no se repitan, ya que esto puede comprometer la confidencialidad del cifrado. Por último opino que en producción se deben utilizar implementaciones confiables como AES-CTR o ChaCha20, evitando diseñar algoritmos propios que puedan contener fallos de seguridad.

## Parte 3: Validación y Pruebas (10 puntos)

## 3.1 Ejemplos de Entrada/Salida
### Incluya en su documentación al menos 3 ejemplos que muestren:
- Texto plano original
- Texto cifrado (puede mostrarse en formato hexadecimal o base64)
- Texto descifrado
- Clave utilizada

In [23]:
clave = "cifrado"

mensaja_a_cifrar ="un cifrado de informacion muy generico"  

print("mensaje a cifrar es el siguiente: ",mensaja_a_cifrar)

cifrado = cifrar(mensaja_a_cifrar, clave )

print( "mensaje a cifrado en bytes", cifrado)

print("El mensaje a descifrar es el siguiente: ",descifrar(cifrado, clave ))

print("la clave a utilizar es: ", clave)

mensaje a cifrar es el siguiente:  un cifrado de informacion muy generico
mensaje a cifrado en bytes b's\x19\xd4f\xaa\xe2\x13\xf6hlc-X\x0b\xcf^\xf8i\xca^\xa5\xcdS\xfa8\xdd\x87M\xb9d\xf6\xcag\xfb\xebK&\x89'
El mensaje a descifrar es el siguiente:  un cifrado de informacion muy generico
la clave a utilizar es:  cifrado


In [24]:
clave = "cesarcifrado"

mensaja_a_cifrar ="La prueba definita se utiliza con el cifrado de cesar" 

print("mensaje a cifrar es el siguiente: ",mensaja_a_cifrar)

cifrado = cifrar(mensaja_a_cifrar, clave )

print( "mensaje a cifrado en bytes", cifrado)

print("El mensaje a descifrar es el siguiente: ",descifrar(cifrado, clave ))

print("la clave a utilizar es: ", clave)

mensaje a cifrar es el siguiente:  La prueba definita se utiliza con el cifrado de cesar
mensaje a cifrado en bytes b'I}\x9c\x04\xb5\x93\xa3g\x1c\xa5\xc0\xf9\xeb@\xf4\x8a\xe4R\xca:\xeb\t\xcc\xe8I\xf8\xee\x93\xd8\xc4\x1b\xe7\xe8\x8a\xc2~\x90W\xb4#\xb5\xc9[1r\xfdx\xa7 ,`\xae\xab'
El mensaje a descifrar es el siguiente:  La prueba definita se utiliza con el cifrado de cesar
la clave a utilizar es:  cesarcifrado


In [25]:
clave = "pruebautilizadoenISO129"

mensaja_a_cifrar ="Que iso se debe de utilizar para el uso de encriptacion de datos" 

print("mensaje a cifrar es el siguiente: ",mensaja_a_cifrar)

cifrado = cifrar(mensaja_a_cifrar, clave )

print( "mensaje a cifrado en bytes", cifrado)

print("El mensaje a descifrar es el siguiente: ",descifrar(cifrado, clave ))

print("la clave a utilizar es: ", clave)

mensaje a cifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos
mensaje a cifrado en bytes b"\x9f\x1d-\x9e\xa9~\xf4\x82\xb4#\x95\x1b\x89W\xdf}a\x0c\xf5Q\xc7\x8aZQ\xe0\xc1\xd6\xe4\x1c\x9a\x1eT\xbe)\xe8[\x00\xdb\xe8\xdbk\x80\x0f\x17\xe2\x86\x1f`\x84\xa6$\x11\xbbH'\xd5P[)|\x0e\xa4\x98\xef"
El mensaje a descifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos
la clave a utilizar es:  pruebautilizadoenISO129


## 3.2 Pruebas Unitarias
### Implemente pruebas que validen:
- El descifrado recupera exactamente el mensaje original
- Diferentes claves producen diferentes textos cifrados
- La misma clave produce el mismo texto cifrado (determinismo)
- El cifrado maneja correctamente mensajes de diferentes longitudes

In [26]:
#El descifrado recupera exactamente el mensaje original

clave = "pruebautilizadoenISO129"

mensaja_a_cifrar ="Que iso se debe de utilizar para el uso de encriptacion de datos" 

print("mensaje a cifrar es el siguiente: ",mensaja_a_cifrar)

cifrado = cifrar(mensaja_a_cifrar, clave )

print("El mensaje a descifrar es el siguiente: ",descifrar(cifrado, clave ))


mensaje a cifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos
El mensaje a descifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos


In [27]:
#Diferentes claves producen diferentes textos cifrados

clave = "pruebautilizadoenISO129"

mensaja_a_cifrar ="Que iso se debe de utilizar para el uso de encriptacion de datos" 

print("mensaje a cifrar es el siguiente: ",mensaja_a_cifrar)


cifrado = cifrar(mensaja_a_cifrar, clave )


clave = "otraclave"

print("El mensaje a descifrar es el siguiente: ",descifrar(cifrado, clave ))


mensaje a cifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos
El mensaje a descifrar es el siguiente:  ÝV½ðpYÎsÔàÛ²ZFúý³3Y>
&±ú¡7ìøaDÀ¬­-üþûã{J ÌõF}5ð;ç»¥ë


In [29]:
#La misma clave produce el mismo texto cifrado (determinismo)

clave = "pruebautilizadoenISO129"

mensaja_a_cifrar ="Que iso se debe de utilizar para el uso de encriptacion de datos" 

print("mensaje a cifrar es el siguiente: ",mensaja_a_cifrar)


cifrado = cifrar(mensaja_a_cifrar, clave )

print("El mensaje a descifrar es el siguiente: ",descifrar(cifrado, clave ))


clave2 = "pruebautilizadoenISO129"

mensaja_a_cifrar2 ="Que iso se debe de utilizar para el uso de encriptacion de datos" 

print("mensaje a cifrar es el siguiente: ",mensaja_a_cifrar2)


cifrado = cifrar(mensaja_a_cifrar2, clave2 )

print("El mensaje a descifrar es el siguiente: ",descifrar(cifrado, clave2 ))


mensaje a cifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos
El mensaje a descifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos
mensaje a cifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos
El mensaje a descifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos


In [ ]:
#La misma clave produce el mismo texto cifrado (determinismo)

clave = "di"

mensaja_a_cifrar ="Que iso" 

print("mensaje a cifrar es el siguiente: ",mensaja_a_cifrar)


cifrado = cifrar(mensaja_a_cifrar, clave )

print("El mensaje a descifrar es el siguiente: ",descifrar(cifrado, clave ))


clave2 = "di"

mensaja_a_cifrar2 ="Que iso se debe de utilizar para el uso de encriptacion de datos" 

print("mensaje a cifrar es el siguiente: ",mensaja_a_cifrar2)


cifrado = cifrar(mensaja_a_cifrar2, clave2 )

print("El mensaje a descifrar es el siguiente: ",descifrar(cifrado, clave2 ))


mensaje a cifrar es el siguiente:  Que iso 
El mensaje a descifrar es el siguiente:  Que iso 
mensaje a cifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos
El mensaje a descifrar es el siguiente:  Que iso se debe de utilizar para el uso de encriptacion de datos
